## Metrics Calculations Tutorial

This notebook provides examples on how to carry out data metrics calcuations and analysis using the post_processing python library. Be sure to go through the [Quick Start](https://nhs-postprocessing.readthedocs.io/en/stable/QuickStart.html) section of the [documentation](https://nhs-postprocessing.readthedocs.io/en/stable/index.html) for instructions on how to access and import the libary and its packages.

If you would like to open an editable runnable version of the tutorial click [here](https://mybinder.org/v2/gh/UchechukwuUdenze/NHS_PostProcessing/main?%2FHEAD=&urlpath=%2Fdoc%2Ftree%2Fdocs%2Fsource%2Fnotebooks%2Ftutorial-metrics.ipynb) to be directed to a binder platform

<mark>The Library is still under active development and empty sections will be completed in Due time</mark>

### Table of content
- [Available Metrics](#available-metrics)
- [Single Data Metrics](#single-data-metrics)
- [Comparison Metrics](#comparison-metrics)

 All files are available in the github repository [here](https://github.com/UchechukwuUdenze/NHS_PostProcessing/tree/main/docs/source/notebooks)

### Requirements

The conda environmnent contains all libraries associated the post processing library. After setting up the conda environment, you only have to import the metrics maniupulation module from postprocessinglib.evaluation.

In [1]:
### Remove and modify these later.
import sys
import pandas as pd
sys.path.append("../../../")

In [2]:
from postprocessinglib.evaluation import data, metrics

Lets use one of the data blocks from the data manipulation tutorial

In [ ]:
# passing a controlled csv file for testing
path_output = "MESH_output_streamflow_2.csv"
path_input = "Station_data.xlsx"

DATAFRAMES = data.generate_dataframes(csv_fpath=path_output, warm_up=91)
               
Stations = pd.read_excel(io=path_input)

ignore = []
for i in range(0, len(Stations)):
    if Stations['Properties'][i] == 'X':
        ignore.append(i)

Stations = Stations.drop(Stations[Stations['Properties'] == 'X'].index)
Stations = Stations.set_index('Station Number')

for i in reversed(ignore):
        DATAFRAMES["DF_OBSERVED"] = DATAFRAMES["DF_OBSERVED"].drop(columns = DATAFRAMES['DF_OBSERVED'].columns[i])
        DATAFRAMES['DF_SIMULATED']  = DATAFRAMES["DF_SIMULATED"].drop(columns = DATAFRAMES['DF_SIMULATED'].columns[i])
        for key, dataframe in DATAFRAMES.items():
            if key != "DF_SIMULATED" and key != "DF_OBSERVED":
                DATAFRAMES[key] = dataframe.drop(columns = dataframe.columns[[2*i, 2*i+1]])
            

# for key, value in DATAFRAMES.items():
#     print(f"{key}:\n{value.head}")

The start date for the Observed Data is 1982-01-01
The start date for the Simulated Data is 1982-01-01
The start date for the Merged Data is 1982-01-01


Now that we have our data, let's jump right in!

### Available Metrics

Because the library is in active development, there will be regular removals and additions to its features. As a rule of thumb therefore it is always a good idea to check what it can do at the time of use. We can do this by going ->

In [4]:
metrics.available_metrics()

['MSE - Mean Square Error',
 'RMSE - Roor Mean Square Error',
 'MAE - Mean Average Error',
 'NSE - Nash-Sutcliffe Efficiency ',
 'NegNSE - Nash-Sutcliffe Efficiency * -1',
 'LogNSE - Log of Nash-Sutcliffe Efficiency',
 'NegLogNSE - Log of Nash-Sutcliffe Efficiency * -1',
 'KGE - Kling-Gupta Efficiency',
 'NegKGE - Kling-Gupta Efficiency * -1',
 'KGE 2012 - Kling-Gupta Efficiency modified as of 2012',
 'BIAS- Prcentage Bias',
 'AbsBIAS - Absolute Value of the Percentage Bias',
 'TTP - Time to Peak',
 'TTCoM - Time to Centre of Mass',
 'SPOD - Spring Pulse ONset Delay',
 'FDC Slope - Slope of the Flow Duration Curve']

### Single Data Metrics
These are the metrics that only apply to just one of either the simulated or observed data. They are less about analysis and more about obtaining information about the data. These aren't made to compare but rather to inform trends and behaviours at a particular station. The library has 4 of them :

- [Time to Peak](#time-to-peak)
- [Time to Centre of Mass](#time-to-centre-of-mass)
- [Spring Pulse Onset Delay](#spring-pulse-onset-delay)
- [Slope of the Flow Duration Curve](#flow-duration-curve-slope)

#### Time to Peak
This helps to show how long it takes on average to get to the highest streamflow each year. An example is shown below:

In [5]:
# The Time to Peak for the simulated data will look like 
print(metrics.time_to_peak(df=DATAFRAMES['DF_SIMULATED']))

# The time to peak for the observed data looks like:-
print(metrics.time_to_peak(df=DATAFRAMES['DF_OBSERVED']))

[170.0, 177.0, 176.0, 168.0, 171.0, 175.0, 166.0, 156.0, 156.0, 170.0, 179.0, 162.0, 171.0, 171.0, 168.0, 175.0, 170.0, 190.0, 187.0, 184.0, 187.0, 174.0, 173.0, 214.0, 176.0, 184.0, 149.0, 148.0, 155.0, 186.0, 141.0, 143.0, 154.0, 171.0, 177.0, 170.0, 172.0, 178.0, 147.0, 155.0]
[157.0, 157.0, 158.0, 159.0, 160.0, 172.0, 175.0, 166.0, 165.0, 173.0, 189.0, 169.0, 164.0, 169.0, 167.0, 167.0, 171.0, 163.0, 169.0, 159.0, 156.0, 184.0, 178.0, 179.0, 184.0, 174.0, 119.0, 115.0, 121.0, 172.0, 128.0, 123.0, 134.0, 123.0, 158.0, 111.0, 150.0, 152.0, 161.0, 139.0]


As you can see, at the first station, on average, over the years, the highest predicted streamflow value will usually occur after 170 days - somewhere in the third week of June. For the second station on average, over the years, the highest predicted streamflow value usually occur after 177 days - somewhere in the final week of June. 
As you can see, you are able to observe and notice trends with the data at specific stations.

#### Time to Centre of Mass
This helps to show how long it takes on average to obtain 50% of the streamflow each year. An example is shown below:

In [6]:
# The Time to Centre of Mass for the simulated data will look like 
print(metrics.time_to_centre_of_mass(df=DATAFRAMES['DF_SIMULATED']))

# The time to Centre of Mass for the observed data looks like:-
print(metrics.time_to_centre_of_mass(df=DATAFRAMES['DF_OBSERVED']))

[184.0, 166.0, 188.0, 182.0, 182.0, 183.0, 190.0, 183.0, 180.0, 158.0, 175.0, 185.0, 169.0, 181.0, 182.0, 181.0, 187.0, 175.0, 190.0, 190.0, 192.0, 187.0, 193.0, 205.0, 189.0, 190.0, 152.0, 147.0, 161.0, 189.0, 155.0, 160.0, 167.0, 169.0, 187.0, 171.0, 190.0, 187.0, 156.0, 183.0]
[177.0, 179.0, 183.0, 178.0, 180.0, 178.0, 199.0, 194.0, 186.0, 172.0, 204.0, 186.0, 177.0, 190.0, 187.0, 177.0, 194.0, 172.0, 182.0, 177.0, 180.0, 195.0, 200.0, 184.0, 202.0, 183.0, 150.0, 132.0, 138.0, 183.0, 154.0, 149.0, 146.0, 157.0, 181.0, 150.0, 187.0, 173.0, 179.0, 179.0]


As you can see, at the first station, on average, over the years, 50% of the total volume of streamflow each year will usually have occured by 177 days - somewhere in the final week of June and for the second stations, after 179 days - Right at the end of June. 

#### Spring Pulse Onset Delay
This is used to determine what day snowmelt starts. An example is shown below:

In [7]:
# The Spring Pulse Onset for the simulated data will look like 
print(metrics.SpringPulseOnset(df=DATAFRAMES['DF_SIMULATED']))

# The Spring Pulse Onset for the observed data looks like:-
print(metrics.SpringPulseOnset(df=DATAFRAMES['DF_OBSERVED']))

[127.0, 127.44117647058823, 116.0, 115.70588235294117, 127.0, 126.67647058823529, 119.0, 118.91176470588235, 121.0, 121.29411764705883, 124.0, 124.08823529411765, 140.0, 139.7941176470588, 127.0, 126.73529411764706, 126.0, 125.97058823529412, 296.0, 296.5, 113.0, 113.38235294117646, 135.0, 135.11764705882354, 110.0, 109.6470588235294, 123.0, 123.32352941176471, 126.0, 125.97058823529412, 132.0, 131.94117647058823, 128.0, 127.73529411764706, 123.0, 122.52941176470588, 144.0, 143.8235294117647, 145.0, 145.1764705882353, 145.0, 144.88235294117646, 121.0, 121.41176470588235, 120.0, 119.88235294117646, 196.0, 196.14705882352942, 137.0, 137.26470588235293, 121.0, 121.23529411764706, 107.0, 106.8529411764706, 97.5, 97.52941176470588, 106.0, 105.97058823529412, 117.0, 116.88235294117646, 112.0, 111.70588235294117, 107.0, 106.8529411764706, 112.0, 112.17647058823529, 108.0, 107.88235294117646, 116.0, 115.91176470588235, 126.0, 125.88235294117646, 204.0, 204.47058823529412, 136.0, 136.2352941176

C:\Users\udenzeU\AppData\Local\anaconda3\envs\postprocessing\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\udenzeU\AppData\Local\anaconda3\envs\postprocessing\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[113.0, 112.53125, nan, nan, 101.0, 101.09677419354838, 114.0, 114.0, 115.0, 114.96969696969697, 124.0, 124.23529411764706, 142.0, 141.8, 136.0, 136.44117647058823, 138.0, 138.3235294117647, 296.0, 296.4516129032258, 172.0, 171.5, 139.0, 138.7941176470588, 118.0, 118.29411764705883, nan, nan, nan, nan, 156.0, 155.7941176470588, 135.0, 134.76470588235293, 101.0, 100.87878787878788, 112.0, 112.08823529411765, 103.0, 102.79411764705883, 107.0, 107.17647058823529, 138.0, 138.14705882352942, nan, nan, 140.0, 139.5, nan, nan, 106.0, 105.58823529411765, nan, nan, nan, nan, nan, nan, 101.0, 101.0909090909091, 93.5, 93.45454545454545, nan, nan, nan, nan, 89.5, 89.5, 98.4, 98.3529411764706, 69.3, 69.3125, 206.0, 205.55172413793105, 126.0, 126.02941176470588, 108.0, 107.82352941176471, 99.4, 99.4375]


This shows us that at the first station, on average, over the years, snowmelt is predicted to begin 127 days into the year - somewhere in the First week of May. For the second station on average, over the years, snowmelt is predicted to begin 127.44 days into the year - somewhere in the First week of May as well

#### Flow Duration Curve Slope
This is used to calculate the slope of the flow duration curve. An example is shown below:

In [8]:
# The Fliw Duration Curve for the Simulated Data will look like 
print(metrics.slope_fdc(df=DATAFRAMES['DF_SIMULATED']))

# You can also specify which percentile to pick values from 
print(metrics.slope_fdc(df=DATAFRAMES['DF_SIMULATED'], percentiles=[25, 77]))

[3.1504, 2.376, 2.1143, 4.2863, 3.7256, 2.7546, 8.0229, 7.094, 1.5833, 4.0913, 6.9928, 2.3851, 5.78, 2.7093, 2.6374, 2.3301, 5.65, 5.627, 1.1424, 1.3281, 1.5908, 6.1167, 0.7576, 0.71, 6.3471, 1.0788, 4.4442, 3.1001, 5.1716, 1.2852, 5.2684, 4.5245, 3.5924, 3.9921, 1.7148, 5.9512, 1.3825, 0.6681, 5.2883, 1.133]
[2.6692, 2.9633, 2.1872, 3.6614, 3.5067, 2.9249, 7.4631, 6.2483, 2.0158, 5.0284, 6.8476, 2.4823, 6.2091, 2.8819, 2.6386, 2.6111, 5.2336, 5.872, 1.54, 1.795, 2.0136, 5.723, 1.3421, 0.6891, 5.7743, 1.5009, 4.7834, 3.1828, 5.7113, 1.6334, 5.7889, 4.8883, 3.9734, 3.9872, 1.8532, 5.9303, 1.2939, 0.9673, 6.2699, 1.3136]


### Comparison Metrics

These are the metrics that are used to compare the simulated and observed data. They work to show accurately we are able to predict the streamflow values using the models. Every other metric is a comparison metric. They are shown below:

- [Mean Square Error](#mean-square-error)
- [Root Mean Square Error](#root-mean-square-error)
- [Mean Average Error](#mean-average-error)
- [Nash-Sutcliffe Efficiency](#nash-sutcliffe-efficiency)
- [Kling-Gupta Efficiency](#kling-gupta-efficiency)
- [Percentage Bias](#percentage-bias)

#### Mean Square Error


In [9]:
# Mean square error for the data we were given
print(metrics.mse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[1304.0, 801.2, 17.44, 5539.0, 4951.0, 14250.0, 85.48, 577.5, 1838.0, 45.26, 87.61, 1898.0, 535.5, 4722.0, 6484.0, 4763.0, 621.8, 125.1, 1415.0, 2282.0, 3487.0, 798.2, 11150.0, 1876.0, 1287.0, 13730.0, 54.8, 8.375, 41.19, 17410.0, 19.85, 89.69, 166.6, 170.6, 20190.0, 36.27, 21630.0, 68600.0, 2450.0, 139600.0]


#### Root Mean Square Error

In [10]:
# Root Mean square error for the data we were given
print(metrics.rmse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[36.11, 28.31, 4.177, 74.42, 70.36, 119.4, 9.246, 24.03, 42.87, 6.728, 9.36, 43.57, 23.14, 68.72, 80.53, 69.01, 24.94, 11.19, 37.61, 47.77, 59.05, 28.25, 105.6, 43.32, 35.87, 117.2, 7.403, 2.894, 6.418, 131.9, 4.456, 9.471, 12.91, 13.06, 142.1, 6.023, 147.1, 261.9, 49.5, 373.6]


#### Mean Average Error

In [11]:
# Mean Average error for the data we were given
print(metrics.mae(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[16.84, 6.988, 1.243, 38.55, 35.56, 73.62, 6.776, 17.88, 36.54, 5.124, 4.126, 36.22, 10.51, 46.69, 55.0, 45.74, 12.17, 4.029, 18.92, 22.68, 27.11, 12.24, 59.89, 33.12, 25.81, 68.39, 3.256, 1.057, 2.002, 73.9, 1.532, 4.54, 5.686, 4.938, 77.78, 1.584, 80.23, 165.0, 18.82, 219.5]


#### Nash-Sutcliffe Efficiency

In [12]:
# Nash-Sutcliffe Efficiency for the data we were given
print(metrics.nse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[0.515, -1.746, -2.021, 0.6165, 0.6588, 0.6588, 0.3662, 0.663, 0.3311, -0.3922, -2.092, 0.4639, 0.7037, 0.6905, 0.579, 0.5144, 0.4019, -0.175, 0.588, 0.4829, 0.3291, -0.9311, -0.7638, -1.619, 0.186, 0.3961, -0.881, -0.01691, -1.577, 0.3322, 0.3656, -0.009723, 0.2154, -0.2041, 0.3694, -0.2525, 0.1911, 0.1621, -5.021, -0.1103]


##### Logarithm of the Nash-Sutcliffe Efficiency

In [13]:
# Logarithm of the Nash-Sutcliffe Efficiency for the data we were given
print(metrics.lognse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[-0.2932, -0.1999, -0.0328, 0.1276, 0.1683, -0.1675, -0.4786, -1.66, -1.871, 0.03629, -24.23, -2.014, -1.096, -0.1154, 0.03147, -0.7631, -0.01762, -1.826, -0.02795, 0.05579, 0.2104, -1.478, -1.822, -7.553, -0.8313, -0.9935, -0.5163, -0.06095, 0.03787, -0.3842, -2.067, -0.0781, 0.126, 0.4076, 0.1954, -9.342, 0.3623, 0.2231, -0.6795, 0.2658]


#### Kling-Gupta Efficiency

In [14]:
# Kling-Gupta Efficiency for the data we were given
print(metrics.kge(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[0.5017, -0.1188, -0.02808, 0.7811, 0.8235, 0.8061, 0.5993, 0.5833, 0.5814, 0.09846, -0.2068, 0.6175, 0.716, 0.7686, 0.7068, 0.6922, 0.5178, 0.3649, 0.7552, 0.6901, 0.6474, -0.01263, 0.08817, 0.0335, 0.4339, 0.6064, 0.2364, 0.5008, 0.07385, 0.6284, 0.2052, 0.214, 0.2362, 0.4282, 0.6415, -0.1692, 0.6154, 0.6054, -0.8191, 0.4612]


##### Modified Kling Gupta efficiency
This is different from the regular kge in that this uses the coefficient of Variation as its bias term (i.e., std/mean) as opposed to just the mean

In [15]:
# Kling-Gupta Efficiency for the data we were given
print(metrics.kge_2012(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[0.5565, 0.05954, -0.2323, 0.7562, 0.823, 0.7465, 0.3209, 0.3656, 0.1112, 0.1086, -0.175, 0.2076, 0.5706, 0.6946, 0.7647, 0.5397, 0.6523, -0.2636, 0.6491, 0.6366, 0.5714, 0.1706, -0.08405, 0.009723, 0.2089, 0.4148, -0.06664, 0.5111, 0.01296, 0.4527, 0.2693, -0.08533, 0.001935, 0.2424, 0.525, -0.1853, 0.6101, 0.6152, -0.08051, 0.3938]


#### Percentage Bias

In [16]:
# Percentage Bias for the data we were given
print(metrics.bias(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

[-34.75, 10.66, -14.59, 9.775, 0.3038, -8.02, -20.29, -39.02, -35.32, -48.43, 1.626, -33.74, -24.57, -6.671, 17.39, -13.53, 11.5, -38.02, -12.28, -17.67, -15.92, 10.73, -8.756, -36.17, -13.47, -14.13, -23.32, 18.72, -4.658, -14.94, -56.19, -53.95, -57.12, -23.16, -10.06, -60.68, -2.505, 1.385, 56.78, -5.43]


Now that we have seen individual metrics, we also have the ability to calculate a list of metrics using our **calculate_all_metrics** or **calculate_metrics(list of merics)**. These are shown below:

In [17]:
metrices = ["MSE", "RMSE", "MAE", "NSE", "NegNSE"]
for key, value in metrics.calculate_metrics(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED'],
                                            metrices=metrices).items():
    print(f"{key}:\n {value}")

MSE:
 [1304.0, 801.2, 17.44, 5539.0, 4951.0, 14250.0, 85.48, 577.5, 1838.0, 45.26, 87.61, 1898.0, 535.5, 4722.0, 6484.0, 4763.0, 621.8, 125.1, 1415.0, 2282.0, 3487.0, 798.2, 11150.0, 1876.0, 1287.0, 13730.0, 54.8, 8.375, 41.19, 17410.0, 19.85, 89.69, 166.6, 170.6, 20190.0, 36.27, 21630.0, 68600.0, 2450.0, 139600.0]
RMSE:
 [36.11, 28.31, 4.177, 74.42, 70.36, 119.4, 9.246, 24.03, 42.87, 6.728, 9.36, 43.57, 23.14, 68.72, 80.53, 69.01, 24.94, 11.19, 37.61, 47.77, 59.05, 28.25, 105.6, 43.32, 35.87, 117.2, 7.403, 2.894, 6.418, 131.9, 4.456, 9.471, 12.91, 13.06, 142.1, 6.023, 147.1, 261.9, 49.5, 373.6]
MAE:
 [16.84, 6.988, 1.243, 38.55, 35.56, 73.62, 6.776, 17.88, 36.54, 5.124, 4.126, 36.22, 10.51, 46.69, 55.0, 45.74, 12.17, 4.029, 18.92, 22.68, 27.11, 12.24, 59.89, 33.12, 25.81, 68.39, 3.256, 1.057, 2.002, 73.9, 1.532, 4.54, 5.686, 4.938, 77.78, 1.584, 80.23, 165.0, 18.82, 219.5]
NSE:
 [0.515, -1.746, -2.021, 0.6165, 0.6588, 0.6588, 0.3662, 0.663, 0.3311, -0.3922, -2.092, 0.4639, 0.7037, 0.6

We are also able to save these metrics as text files and csv files by specifying the **format** parameter and even the **out** parameter to specify a name to save it as.

In [18]:
metrics.calculate_metrics(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED'], metrices=metrices,
#                          format='txt', out='metrics'
                         )

{'MSE': [1304.0,
  801.2,
  17.44,
  5539.0,
  4951.0,
  14250.0,
  85.48,
  577.5,
  1838.0,
  45.26,
  87.61,
  1898.0,
  535.5,
  4722.0,
  6484.0,
  4763.0,
  621.8,
  125.1,
  1415.0,
  2282.0,
  3487.0,
  798.2,
  11150.0,
  1876.0,
  1287.0,
  13730.0,
  54.8,
  8.375,
  41.19,
  17410.0,
  19.85,
  89.69,
  166.6,
  170.6,
  20190.0,
  36.27,
  21630.0,
  68600.0,
  2450.0,
  139600.0],
 'RMSE': [36.11,
  28.31,
  4.177,
  74.42,
  70.36,
  119.4,
  9.246,
  24.03,
  42.87,
  6.728,
  9.36,
  43.57,
  23.14,
  68.72,
  80.53,
  69.01,
  24.94,
  11.19,
  37.61,
  47.77,
  59.05,
  28.25,
  105.6,
  43.32,
  35.87,
  117.2,
  7.403,
  2.894,
  6.418,
  131.9,
  4.456,
  9.471,
  12.91,
  13.06,
  142.1,
  6.023,
  147.1,
  261.9,
  49.5,
  373.6],
 'MAE': [16.84,
  6.988,
  1.243,
  38.55,
  35.56,
  73.62,
  6.776,
  17.88,
  36.54,
  5.124,
  4.126,
  36.22,
  10.51,
  46.69,
  55.0,
  45.74,
  12.17,
  4.029,
  18.92,
  22.68,
  27.11,
  12.24,
  59.89,
  33.12,
  25.81,
  68